# Minimalist Tower RTS Agent
Sam Greydanus

See the `minirts-env` notebook in this directory for details on the RTS environment.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch, time, os, glob
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

from towers import Towers
from rtsenv import RTSEnv

## Define a network

In [2]:
class NNPolicy(torch.nn.Module): # an actor-critic neural network
    def __init__(self, num_actions):
        super(NNPolicy, self).__init__()
        self.conv1 = nn.Conv2d(6, 16, 2, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 16, 2, stride=2, padding=1)
        self.conv3 = nn.Conv2d(16, 16, 2, stride=2, padding=1)
        self.flat_dim = flat_dim = 16 * 3 * 3
        self.critic_linear, self.actor_linear = nn.Linear(flat_dim, 1), nn.Linear(flat_dim, num_actions)

    def forward(self, inputs):
        x = F.elu(self.conv1(inputs))
        x = F.elu(self.conv2(x))
        x = F.elu(self.conv3(x))
        hx = x.view(-1, self.flat_dim)
        value, probs = self.critic_linear(hx), F.softmax(self.actor_linear(hx), dim=1)
        return value, probs
    
    def try_load(self, save_dir):
        paths = glob.glob(save_dir + '*.tar') ; rew = None
        if len(paths) > 0:
            ckpts = [float(s.split('_')[-2]) for s in paths]
            ix = np.argmax(ckpts) ; rew = ckpts[ix]
            self.load_state_dict(torch.load(paths[ix]))
        print("\tno saved models") if rew is None else print("\tloaded model: {}".format(paths[ix]))
        return rew

## Define some hyperparameters

In [8]:
class Args():
    pass
args = Args()
args.lr = 1e-3
args.num_actions = 4
args.batch_size = 128
args.total_steps = 20000 # just set this to a big number
args.printevery = 5 # print stats every n seconds
args.saveevery = 30 # save model every n seconds
args.rewardthresh = -0.05
args.noiselevel = 0.05
args.save_dir = './saved/'

os.makedirs(args.save_dir) if not os.path.exists(args.save_dir) else None # make dir to save models etc.

## Explore the environment, train an agent

PyTorch has a great API for reinforcement learning now

In [9]:
model = NNPolicy(num_actions=args.num_actions)
optimizer = torch.optim.RMSprop(model.parameters(), lr=args.lr, momentum=0.9)
run_vloss, run_ploss, run_reward = None, None, None

print('training a simple policy gradient CNN agent on the towers RTS game')
print_t = save_t = time.time()
for step in range(args.total_steps + 1):
    envs = [Towers() for i in range(args.batch_size)]
    obs = np.stack([e.reset() for e in envs])
    state = Variable(torch.Tensor(obs + np.random.randn(*obs.shape)*args.noiselevel))
    value, probs = model(state)
    
    m = torch.distributions.Categorical(probs)
    actions = m.sample()
    raw_reward = [e.step(actions.data.numpy().ravel()[i])[1] for i, e in enumerate(envs)]
    raw_reward = Variable(torch.Tensor(raw_reward))
    norm_reward = (raw_reward - raw_reward.mean())/raw_reward.std() # this sometimes helps convergence

    vloss = 0.5*(value.view(-1) - norm_reward).pow(2).mean()
    ploss = (-m.log_prob(actions) * norm_reward).mean()
    (vloss + ploss).backward()
    optimizer.step()
    optimizer.zero_grad()

    np_vloss = vloss.data.view(-1)[0]
    np_ploss = ploss.data.view(-1)[0]
    np_reward = raw_reward.mean().data.view(-1)[0]

    run_vloss = np_vloss if run_vloss is None else 0.99*run_vloss + 0.01*np_vloss
    run_ploss = np_ploss if run_ploss is None else 0.99*run_ploss + 0.01*np_ploss
    run_reward = np_reward if run_reward is None else 0.99*run_reward + 0.01*np_reward
    
    if time.time() - print_t > args.printevery:
        print_t = time.time()
        print('\tstep {}, value loss {:.3f}, policy loss {:.3f}, reward {:.3f}'
              .format(step, run_vloss, run_ploss, run_reward))
    
    if time.time() - save_t > args.saveevery:
        save_t = time.time()
        paths = glob.glob(args.save_dir + '*.tar') ; rew_best_saved_model = -10000
        if len(paths) > 0:
            ckpts = [float(s.split('_')[-2]) for s in paths]
            ix = np.argmax(ckpts) ; rew_best_saved_model = ckpts[ix]
        if rew_best_saved_model < run_reward:
            name = 'model_step_{:.0f}_reward_{:.3f}_.tar'.format(step,run_reward)
            print('\t\tsaved {}'.format(name))
            torch.save(model.state_dict(), args.save_dir + name)
        else:
            print('\t\tlol your model sucks keep training')
        
    if run_reward > args.rewardthresh:
        print("environment solved")
        name = 'model_step_{:.0f}_reward_{:.2f}_.tar'.format(step,run_reward) ; print('\t\tsaved {}'.format(name))
        torch.save(model.state_dict(), args.save_dir + name)
        break

training a simple policy gradient CNN agent on the towers RTS game
	step 117, value loss 0.435, policy loss -0.019, reward -0.331
	step 240, value loss 0.393, policy loss -0.028, reward -0.265
	step 353, value loss 0.382, policy loss -0.038, reward -0.191
	step 456, value loss 0.368, policy loss -0.041, reward -0.157
	step 562, value loss 0.359, policy loss -0.039, reward -0.143
		saved model_step_658_reward_-0.138_.tar
	step 662, value loss 0.350, policy loss -0.031, reward -0.138
	step 758, value loss 0.351, policy loss -0.027, reward -0.131
	step 843, value loss 0.347, policy loss -0.018, reward -0.131
	step 951, value loss 0.342, policy loss -0.022, reward -0.128
	step 1061, value loss 0.353, policy loss -0.026, reward -0.122
	step 1162, value loss 0.351, policy loss -0.023, reward -0.119
		saved model_step_1249_reward_-0.117_.tar
	step 1254, value loss 0.359, policy loss -0.026, reward -0.118
	step 1359, value loss 0.362, policy loss -0.029, reward -0.113
	step 1455, value loss 0.

KeyboardInterrupt: 

## Try loading a saved model

Yeah, the saving and loading I/O works!

In [10]:
model = NNPolicy(num_actions=args.num_actions)
ckpt_reward = model.try_load(args.save_dir)
print('this saved model had a mean reward of {:.2f}'.format(ckpt_reward))

	loaded model: ./saved/model_step_5090_reward_-0.065_.tar
this saved model had a mean reward of -0.07
